In [102]:
import time
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests

In [103]:
url = " https://food.grab.com/sg/en"

chrome_options = webdriver.ChromeOptions()
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134"
chrome_options.add_argument(f'user-agent={user_agent}')
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(options=chrome_options)
driver.get(url)

KeyboardInterrupt: 

In [ ]:
try:
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Accept")]'))).click()
except TimeoutException:
    pass

In [ ]:
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".ant-layout")))

location_input = driver.find_element(By.ID, 'location-input')
location_input.click()
time.sleep(2)
location_input.clear()

location_input.send_keys("PT Singapore - Choa Chu Kang North 6, Singapore, 689577")

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".ant-btn.submitBtn___2roqB.ant-btn-primary")))

submit_button = driver.find_element(By.CSS_SELECTOR, '.ant-btn.submitBtn___2roqB.ant-btn-primary')
submit_button.click()

In [ ]:
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".ant-layout")))

last_height = driver.execute_script("return document.body.scrollHeight")
t=0
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [ ]:
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "RestaurantListRow___1SbZY")))
layout_div = driver.find_element(By.CLASS_NAME, 'RestaurantListRow___1SbZY')

WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, "name___2epcT")))

<selenium.webdriver.remote.webelement.WebElement (session="8d9729156ed613daf64d50a8c1f61fcb", element="f.D5364A57356C4E0D3E8A46B49E16B1AF.d.A4AE2317DBF30BACF4E390894C934BC0.e.71")>

In [ ]:
ratings = layout_div.find_elements(By.CLASS_NAME, 'numbers___2xZGn')

soup = BeautifulSoup(driver.page_source , 'html.parser')

In [ ]:
details = soup.find_all('div', class_='ant-row-flex ant-row-flex-start ant-row-flex-top asList___1ZNTr')

In [ ]:
import pandas as pd

# Initialize empty lists to store extracted details
restaurant_names = []
restaurant_cuisines = []
ratings = []
direction = []
delivery_times = []
distances = []
discount= []
images = []
promo = []

# Loop through each div element
for div in details:
    # Extract restaurant name
    restaurant_name = div.find('p', class_='name___2epcT').text.strip()
    restaurant_names.append(restaurant_name)
    
    # Extract restaurant cuisine
    cuisine_div = div.find('div', class_='cuisine___T2tCh')
    cuisines = cuisine_div.text.strip() if cuisine_div else None
    
    restaurant_cuisines.append(cuisines)
    
    rating_div = div.find('div', class_='numbersChild___2qKMV')
    if rating_div.find('div', class_='ratingStar'):
        ratings.append(rating_div.text)
    else:
        ratings.append(None)

    directions = div.find_all('div', class_="numbersChild___2qKMV")
    for direction_div in directions:
        if direction_div.find('div', class_="deliveryClock"):
            direction.append(direction_div.text)
    
    discount_div = div.find('div', class_="colInfo___3iLqj").find('div', class_="discount___3h-0m")
    if discount_div: 
        discount.append(discount_div.text)
    else:
        discount.append(None)
    
    image = div.find('img', class_="realImage___2TyNE")
    images.append(image.get('src'))
    
    if div.find('div', class_="promoTagHead___1bjRG"):
        promo.append(True)
    else:
        promo.append(False)
        
    
for x in direction:
        delivery_time, distance  = x.split('•')
        delivery_times.append(delivery_time[:7])
        distances.append(distance[-6:])
        
df = pd.DataFrame({
    'Restaurant Name': restaurant_names,
    'Restaurant Cuisine': restaurant_cuisines,
    'Restaurant Rating': ratings,
    'Restaurant Delivery Time': delivery_times,
    'Restaurant Distance': distances,
    'Discount' : discount,
    'Promo' : promo,
    'Images' : images
})

In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
data = pd.read_csv('data.csv')

# Total count of records
total_count = len(data)

# Null/Non-null statistics
null_non_null_stats = {}
for column in data.columns:
    null_count = data[column].isnull().sum()
    non_null_count = data[column].notnull().sum()
    null_non_null_stats[column] = {'null': null_count, 'non_null': non_null_count}

# Print total count
print(f"Total Count of Records: {total_count}")

# Print null/non-null statistics
print("Null/Non-null Statistics:")
for column, stats in null_non_null_stats.items():
    print(f"Column: {column}, Null: {stats['null']}, Non-null: {stats['non_null']}")

Total Count of Records: 277
Null/Non-null Statistics:
Column: Unnamed: 0, Null: 0, Non-null: 277
Column: Restaurant Name, Null: 0, Non-null: 277
Column: Restaurant Cuisine, Null: 0, Non-null: 277
Column: Restaurant Rating, Null: 42, Non-null: 235
Column: Restaurant Delivery Time, Null: 0, Non-null: 277
Column: Restaurant Distance, Null: 0, Non-null: 277
Column: Discount, Null: 248, Non-null: 29
Column: Promo, Null: 0, Non-null: 277
Column: Images, Null: 0, Non-null: 277


In [ ]:
url_elements = driver.find_elements(By.XPATH, "//a[contains(@href, '/sg/en/restaurant')]")

In [ ]:
ids = [url.get_attribute('href') for url in url_elements if not (url.get_attribute('href').__contains__('search') or url.get_attribute('href').__contains__('category'))]
ids = [id.split('/')[-1][:-1] for id in ids]

In [ ]:
ids = ids[:10] #Indexing down number of ids for testing purposes

In [ ]:
api = f"https://portal.grab.com/foodweb/v2/merchants/"

In [ ]:
url = api + ids[0]
response = requests.get(url)

In [ ]:
response.json()

{'merchant': {'ID': 'SGDD04900',
  'name': "McDonald's - Springleaf Tower",
  'cuisine': 'Burger,Fast Food,Halal',
  'photoHref': 'https://food-cms.grab.com/compressed_webp/merchants/SGDD04900/hero/e82ca4cc15414a5db9db6a1f57cbcd56_1677816676242912044.webp',
  'timeZone': 'Asia/Singapore',
  'ETA': 60,
  'openingHours': {'open': True,
   'displayedHours': '07:00-10:45 11:00-23:59',
   'sun': '7:00am-11:45am 12:00pm-11:59pm',
   'mon': '7:00am-10:45am 11:00am-11:59pm',
   'tue': '7:00am-10:45am 11:00am-11:59pm',
   'wed': '7:00am-10:45am 11:00am-11:59pm',
   'thu': '7:00am-10:45am 11:00am-11:59pm',
   'fri': '7:00am-10:45am 11:00am-1:45am',
   'sat': '7:00am-11:45am 12:00pm-1:45am'},
  'isIntegrated': True,
  'distanceInKm': 893.186,
  'section': 'SGSEC20190911230228023845',
  'latlng': {'latitude': 1.2747543, 'longitude': 103.84613227844237},
  'menu': {'categories': [{'name': "Today's Offer",
     'available': True,
     'items': [{'ID': 'SGITE20240508163209018028',
       'name': 'Chi

In [ ]:
lat, lon = response.json()['merchant']['latlng']['latitude'], response.json()['merchant']['latlng']['longitude']
lat, lon

(1.2747543, 103.84613227844237)

In [104]:
latitudes = []
longitudes = []

for index, id in enumerate(ids):
    url = api + id
    response = requests.get(url)
    lat = response.json()['merchant']['latlng']['latitude']
    lon = response.json()['merchant']['latlng']['longitude']
    latitudes.append(lat)
    longitudes.append(lon)
    time.sleep(5)
    print(lat, lon)

1.2747543 103.84613227844237
1.2841208451045 103.8521381349972
1.284154 103.851789
1.294113995 103.8532129
1.2890973 103.84783774614333
1.2770908240001 103.846051
1.284500731659856 103.8512442
1.284301899 103.8513482
1.291133004612527 103.85015362774963
1.2857211372800634 103.84982956837393


In [117]:
for x in range(len(latitudes)):
    print(f"Restaurant Name - {df.iloc[x]['Restaurant Name']}   Latitude - {latitudes[x]}   Longitude - {longitudes[x]}")

Restaurant Name - Dapur Penyet - North Bridge Road   Latitude - 1.2747543   Longitude - 103.84613227844237
Restaurant Name - McDonald's - Boat Quay   Latitude - 1.2841208451045   Longitude - 103.8521381349972
Restaurant Name - Chuan Taizi Mala Tang - One Raffles Place   Latitude - 1.284154   Longitude - 103.851789
Restaurant Name - Dumpling Darlings - Circular Road   Latitude - 1.294113995   Longitude - 103.8532129
Restaurant Name - WOK HEY - Funan   Latitude - 1.2890973   Longitude - 103.84783774614333
Restaurant Name - Toast Box - Phillip Street   Latitude - 1.2770908240001   Longitude - 103.846051
Restaurant Name - CHICHA San Chen - Change Alley   Latitude - 1.284500731659856   Longitude - 103.8512442
Restaurant Name - Poke Theory - Change Alley Mall   Latitude - 1.284301899   Longitude - 103.8513482
Restaurant Name - Souperstar - One Raffles Place   Latitude - 1.291133004612527   Longitude - 103.85015362774963
Restaurant Name - Burger King - Boat Quay   Latitude - 1.285721137280063